This notebook solves a household problem in a deterministic growth economy using a collocation method.
Model
--
The household chooses its consumption and savings while being constraint by its budget set. Saved capital is used for production next period. Produced goods and leftover capital are returned to the household.

The sequence problem:
$$\underset{ \left(c_t, k_{t+1}\right)_{t=0}^\infty } {max} \sum_{t=0}^\infty \beta^t \, u( c_t )$$
s.t.
$$k_{t+1} = f(k_t) + (1-\delta) k_t - c_t \quad \forall t$$
$$k_0 \, \text{given}$$

can be written recursively as:

$$V(k) = \underset{ c } {max} \, u( c ) + \beta \, V \big( f(k) + (1-\delta) k -c \big)$$
The solution of this Bellman equation is a policy function $c(k)$ for consumption.

Using the first-order condition and the envelope theorem yields:
$$u'(c(k))=\beta \, u'\Big( c\big( f(k) +(1- \delta)k- c(k)\big) \Big) \, \Big( f'\big(f(k) +(1- \delta)k - c(k) \big) + (1-\delta) \Big)\quad\forall k$$
or with invertible utility function
$$c(k)=(u')^{-1}\Bigg( \beta \, u'\Big( c\big(f(k) +(1- \delta)k - c(k)\big) \Big) \Big( f'\big(f(k) + (1- \delta)k - c(k) \big) + (1-\delta) \Big) \Bigg)\quad\forall k$$

Numerical Method
--
The goal of the collocation method is to find an approximate function, $\hat{c}(k)$, that solves the above equation exactly at finitly many $k_i\in[\underline{k}, \bar{k}]$ with $i=1, ..., n$ and is close everywhere else.
This $\hat{c}(k)$ is an element of a family of functions of the form $\sum_{j=1}^m \Phi_j\big(k\big) \, c_j$ and is identified by a vector of coefficients, $c_j$ with $j=1,...,m$. The basis functions, $\Phi_j$, can be Chebyshev polynomials, B-Splines, piecewise linear, etc.

The values of the basis functions at the grid points, $K$, are collected in the matrix $\Psi$, with $\Psi_{ij} = \Phi_j \big(k_i\big)$. 

Finding the coefficients is easy in a ordinary interpolation exercise, where $\hat{c}(K)=\sum_{j=1}^m \Phi_j\big(K\big) \, c_j=\Psi \cdot c=f(K)$ amounts to an matrix inversion. Since $\hat{c}(k)$ appears on the right-hand side of the optimality condition we employ an iterative procedure with naive initial coefficients.

Implementation
--

In [1]:
using Parameters
using BasisMatrices
using LaTeXStrings
using Plots; pyplot();

The model and its functions are:

In [2]:
@with_kw immutable GrowthModel
    β::Float64                           # discount factor
    δ::Float64                           # depreciation of capital
    α::Float64                           # capital share
    A::Float64                           # productivity
    γ::Float64                           # RRA
end

model = GrowthModel(β = 0.95, δ = 0.05, α = 0.3, A = 1, γ = 2)

function steady_state_k(model::GrowthModel)
    @unpack β, α, A, δ = model
    ((1/β-(1-δ))/(A*α))^(1/(α-1))
end

function f(model::GrowthModel, k)
    @unpack α, A, δ = model
    A*k.^α
end

function f_prime(model::GrowthModel, k)
    @unpack α, A, δ = model
    A*α*k.^(α-1)
end

u_crra_prime(c, γ) = c.^-γ
u_crra_prime_inv(u, γ) = u.^(-1/γ)

u_crra_prime_inv (generic function with 1 method)

In [3]:
k_stst = steady_state_k(model)

4.628988089138438

The basis functions of the interpolation are 10 Chebyshev polynomials on a grid that is spanned around the steady state of capital.

In [4]:
basis = Basis(ChebParams(10, 0.2*k_stst, 2*k_stst))

1 dimensional Basis on the hypercube formed by (0.9257976178276875,) × (9.257976178276875,).
Basis families are Cheb


In [5]:
@unpack β, α, A, δ, γ = model
Ψ = BasisMatrix(basis, Expanded()).vals[1]
K = nodes(basis)[1];

In [ ]:
#initial coefficients
a = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0];

stop = false
iteration = 0
a_old = zeros(a)

Kdense = linspace(0.2*k_stst, 2*k_stst, 200)
plot(Kdense, BasisMatrix(basis, Expanded(), Kdense).vals[1] * a, label = iteration)

while !stop
     
    a_old = a
    #right-hand side of the optimality condition
    Y = u_crra_prime_inv(β*u_crra_prime(BasisMatrix(basis, Expanded(), f(model,K) + (1-δ)*K - Ψ*a).vals[1]*a, γ) .* 
            (f_prime(model, f(model,K) + (1-δ)*K - Ψ*a) + 1-δ), γ)
    
    a = Ψ \ Y
        
    if maximum(abs.(u_crra_prime(Y, γ)./u_crra_prime(Ψ*a_old, γ) - 1)) < 1e-10
        stop = true
    end
    
    iteration += 1
    if (iteration % 10 == 0) 
        plot!(Kdense, BasisMatrix(basis, Expanded(), Kdense).vals[1] * a, label = iteration)
    end
end

plot!(Kdense, BasisMatrix(basis, Expanded(), Kdense).vals[1] * a, label = iteration)

In [ ]:
plot(Kdense,BasisMatrix(basis, Expanded(), Kdense).vals[1]*a, ylab="consumption", xlab="capital", label = "consumption policy")

Accuracy
--
We determine the accuracy of the approxiamte policy function $\hat{c}(k)$ by means of the relative approximation error $\mu(k)$. It measures by how many percent the consumption is off from the truth. 
$$u'\Big(\hat{c}(k)\big(1-\mu(k)\big)\Big) = \beta \, u'\Big( \hat{c}\big(f(k) + (1- \delta)k - \hat{c}(k)\big) \Big) \Big( f'\big(f(k) + (1- \delta)k - \hat{c}(k) \big) + (1-\delta) \Big)$$

$$\mu(k)=1-\frac{(u')^{-1}\Bigg( \beta \, u'\Big( \hat{c}\big(f(k) + (1- \delta)k - \hat{c}(k)\big) \Big) \Big( f'\big(f(k) + (1- \delta)k - \hat{c}(k) \big) + (1-\delta) \Big) \Bigg)}{\hat{c}(k)}$$

This means that a positive error of $0.01$ indicates that the household consumes 1 percent too many goods.

In [ ]:
#evaluate on a broader grid
Kdense = linspace(0.05*k_stst, 2.5*k_stst, 200)

#consumption on a dense grid of capital
Cdense = BasisMatrix(basis, Expanded(), Kdense).vals[1]*a

μdense = 1 - u_crra_prime_inv(β*u_crra_prime(BasisMatrix(basis, Expanded(), f(model,Kdense) + (1-δ)*Kdense - Cdense).vals[1] * a, γ) .* 
        (f_prime(model, f(model,Kdense) + (1-δ)*Kdense - Cdense) + 1-δ), γ) ./ Cdense

plot(Kdense, μdense,  label = "EE error")
vline!([0.2*k_stst 2*k_stst], c=:orange, label = [L"$\underbar{k}$" L"$\bar{k}$"])

Within $\underline{k}$ and $\bar{k}$ the approximate policy function solves the Euler Equation very accurately, but even outside the solution is not too bad. An agent with almost no capital would only consume 5 percent too much compared to the optimal policy.

Performance
--

In [ ]:
using Benchmarks

function solve_collocation(model)
    @unpack β, α, A, δ, γ = model

    k_stst = steady_state_k(model)

    basis = Basis(ChebParams(10, 0.2*k_stst, 2*k_stst))

    Ψ = BasisMatrix(basis, Expanded()).vals[1]
    K = nodes(basis)[1]

    a = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    a_old = zeros(a)
    
    stop = false
    
    while !stop

        a_old = a
    
        Y = u_crra_prime_inv(β*u_crra_prime(BasisMatrix(basis, Expanded(), f(model,K) + (1-δ)*K - Ψ*a).vals[1]*a, γ) .* 
                (f_prime(model, f(model,K) + (1-δ)*K - Ψ*a) + 1-δ), γ)

        a = Ψ \ Y

        if maximum(abs.(u_crra_prime(Y, γ)./u_crra_prime(Ψ*a_old, γ) - 1)) < 1e-10
            stop = true
        end
    end
end

@benchmark solve_collocation(model)